In [23]:
import pandas as pd
from sodapy import Socrata


client = Socrata("data.melbourne.vic.gov.au", None)


CPBL = client.get("vh2v-4nfs", limit=2000)  #CPBL:CarParkBayLocation
CPBL_df = pd.DataFrame.from_records(CPBL)

CPBR = client.get("ntht-5rk7", limit=20000)#CPBR:Car Park Bay Restrictions
CPBR_df = pd.DataFrame.from_records(CPBR)


#*****
merge_df=pd.merge(CPBL_df,CPBR_df,left_on='bay_id',right_on='bayid', how='left')
merge_df.isnull().any()
#merge_df.info()
df=merge_df.iloc[:,[0,1,2,4,5,9,10,11,12,13,16,17,18]]
df[['lat','lon','disabilityext1','duration1']].apply(pd.to_numeric)
#df.head(5)
df.describe()
#*******

,bay_id,st_marker_id,status,lat,lon,description1,disabilityext1,duration1,effectiveonph1,endtime1,starttime1,today1,typedesc1
count,909,909,909,909,909,908,908,908,908,908,908,908,908
unique,909,909,2,909,909,76,9,10,2,13,8,3,24
top,2665,C670,Unoccupied,-37.80844946288812,144.952092831316,4P MTR M-F 7:30-18:30,480,240,0,18:30:00,07:30:00,5,4P Meter
freq,1,1,716,1,1,166,313,316,880,622,860,465,244


# Build Map

In [24]:
import folium
import webbrowser
# add pop-up text to each marker on the map
latitude = -37.82
longitude = 144.95

# Create map and display it
map = folium.Map([latitude, longitude], zoom_start=14)

# Display the map of Melbourne
#map
# Instantiate a feature group for the incidents in the dataframe
latitudesU = list(df[df['status']=='Unoccupied'].lat)
longitudesU = list(df[df['status']=='Unoccupied'].lon)
#incidents = folium.map.FeatureGroup()
labelsU = list(df[df['status']=='Unoccupied'].status)

for lat, lng, label in zip(latitudesU, longitudesU, labelsU):
    status_map=folium.Marker(
    location=[lat, lng],
    popup=label,
    icon=folium.Icon(color='blue', icon='ok-sign'),
    control_scale=True
    ).add_to(map)    

latitudesP = list(df[df['status']=='Present'].lat)
longitudesP = list(df[df['status']=='Present'].lon)
#incidents = folium.map.FeatureGroup()
labelsP = list(df[df['status']=='Present'].status)

for lat, lng, label in zip(latitudesP, longitudesP, labelsP):
    status_map=folium.Marker(
    location=[lat, lng],
    popup=label,
    icon=folium.Icon(color='Red', icon='info-sign'),
    control_scale=True 
    ).add_to(map)    


# add incidents to map
#status_map=map.add_child(incidents)


D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: UserWarning: color argument of Icon should be one of: {'purple', 'cadetblue', 'darkred', 'darkpurple', 'red', 'beige', 'lightred', 'green', 'white', 'black', 'darkgreen', 'lightgreen', 'darkblue', 'gray', 'pink', 'orange', 'lightblue', 'blue', 'lightgray'}.


## Map Show

In [25]:
import os 

geo_name = 'Geo_Map.html'
#to save it in a file
status_map.save('images/'+geo_name)
webbrowser.open('images/'+geo_name)

image_path = "file://"+os.getcwd()+ '/'+ 'images/'+geo_name
webbrowser.open(image_path)



True